# With Tool Prompt

In [29]:
import re
import yaml
import json
from openai import OpenAI

In [13]:
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama',
)

In [15]:
weatherTool = {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
        },
        "required": ["location"],
    },
}

In [16]:
toolPrompt = f"""
You have access to the following functions:

Use the function '{weatherTool["name"]}' to '{weatherTool["description"]}':
{json.dumps(weatherTool)}

If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{{\"example_name\": \"example_value\"}}</function>

Reminder:
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- If there is no function call available, answer the question like normal with your current knowledge and do not tell the user about function calls
"""

In [22]:
messages = [
  	{
        "role": "system",
        "content": toolPrompt,
    },
    {
        "role": "user",
        "content": "What is the weather in Tokyo?",
    },
    
]

In [23]:
response = client.chat.completions.create(
    model="llama3.1",
    messages=messages,
    max_tokens=1024,
    temperature=0,
)

messages.append(response.choices[0].message)
print(response.choices[0].message.content)

<function=get_current_weather>{"location": "Tokyo, Japan"}</function>


In [26]:
def parse_tool_response(response: str):
    function_regex = r"<function=(\w+)>(.*?)</function>"
    match = re.search(function_regex, response)

    if match:
        function_name, args_string = match.groups()
        try:
            args = json.loads(args_string)
            return {
                "function": function_name,
                "arguments": args,
            }
        except json.JSONDecodeError as error:
            print(f"Error parsing function arguments: {error}")
            return None
    return None

parsed_response = parse_tool_response(response.choices[0].message.content)
print(parse_tool_response(response.choices[0].message.content))

{'function': 'get_current_weather', 'arguments': {'location': 'Tokyo, Japan'}}


In [27]:
def get_current_weather(location: str) -> str:
    # This would be replaced by a weather API
    if location == "Tokyo, Japan":
        return "62 degrees and cloudy"
    elif location == "Philadelphia, PA":
        return "83 degrees and sunny"
    return "Weather is unknown"

if parsed_response:
    available_functions = {"get_current_weather": get_current_weather}
    function_to_call = available_functions[parsed_response["function"]]
    weather = function_to_call(parsed_response["arguments"]["location"])
    messages.append(
        {
            "role": "tool",
            "content": weather,
        }
    )
    print("Weather answer is: ", weather)

    res = client.chat.completions.create(
        model="llama3.1",
        messages=messages,
        max_tokens=1000,
        temperature=0,
    )
    print("Answer from the LLM: ", res.choices[0].message)
else:
    print("No function call found in the response")

Weather answer is:  62 degrees and cloudy
Answer from the LLM:  ChatCompletionMessage(content='<function=get_current_weather>{"location": "Tokyo, Japan"}</function>', refusal=None, role='assistant', function_call=None, tool_calls=None)
